This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
#pip install python-dotenv

In [2]:
# get api key from your .env file
import os
import dotenv
from dotenv import load_dotenv

#load_dotenv()
load_dotenv('../NASDAQ_API_KEY.env')
API_KEY = os.getenv('NASDAQ_API_KEY')

#print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to use Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

In [3]:
#pip install requests

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [4]:
# First, import the relevant modules
import requests
import json
from datetime import datetime
import numpy as np

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [5]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned

#GET https://data.nasdaq.com/api/v3/datasets/{database_code}/{dataset_code}/data.{return_format}
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?api_key={}".format(API_KEY)
r = requests.get(url)

In [6]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

json_data = r.json()
#Getting an overall picture of the structure of the json
print(json_data.keys())

dict_keys(['dataset_data'])


In [7]:
#A single key ('dataset_data') dictionary with 10 nested keys
print(json_data['dataset_data'].keys())
j_data = json_data['dataset_data']

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])


In [8]:
#Exploring the columns (features) available
j_data['column_names']

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Change',
 'Traded Volume',
 'Turnover',
 'Last Price of the Day',
 'Daily Traded Units',
 'Daily Turnover']

In [9]:
#Example of a daily record of Carl Zeiss Meditec stock in the Frankfurt Stock Exhange (FSE) 
j_data['data'][0]

['2020-12-01',
 112.2,
 112.2,
 111.5,
 112.0,
 None,
 51.0,
 5703.0,
 None,
 None,
 None]

In [10]:
#Converting the key as a datetime object
d = datetime.strptime(j_data['data'][0][0], '%Y-%m-%d')
print(d.year)

2020


### These are your tasks for this mini project:

**1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).**

In [11]:
d_2017 = []
for i in j_data['data']:
    year = datetime.strptime(i[0], '%Y-%m-%d').year
    #Select only records from 2017 in a list
    if year == 2017:
        d_2017.append(i)
#print(d_2017)

**2. Convert the returned JSON object into a Python dictionary.**

In [12]:
#Convert the 2017 record list to a dictionary
data_2017 = {x[0]: x[1:] for x in d_2017}
#print(data_2017)

**3. Calculate what the highest and lowest opening prices were for the stock in this period.**

In [13]:
open_p = [] #List for gwtting the opening prices
diff_close = [] #List for getting the difference between two days
avg_dvol = [] #List for getting the average daily volume
for i in data_2017.keys():
    if data_2017[i][0] != None:
        open_p.append(data_2017[i][0])
    diff_close.append(data_2017[i][3])
    avg_dvol.append(data_2017[i][5])
print('The highest opening price in 2017 was: {} USD'.format(max(open_p)))

The highest opening price in 2017 was: 53.11 USD


**4. What was the largest change in any one day (based on High and Low price)?**

In [14]:
print('The lowest opening price in 2017 was: {} USD'.format(min(open_p)))

The lowest opening price in 2017 was: 34.0 USD


**5. What was the largest change between any two days (based on Closing Price)?**

In [15]:
lar_diff = []
for i in range(len(diff_close)-1):
    lar_diff.append(abs(diff_close[i+1]-diff_close[i]))
print('The largest change between any two days was: {:.2f} USD'.format(max(lar_diff)))

The largest change between any two days was: 2.56 USD


**6. What was the average daily trading volume during this year?**

In [16]:
print('The avergae trading volume in 2017 was: {:.2f}'.format(np.mean(avg_dvol)))

The avergae trading volume in 2017 was: 89124.34


**7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)**

In [17]:
print('The average trading volume in 2017 was: {:.2f}'.format(np.median(avg_dvol)))

The average trading volume in 2017 was: 76286.00
